In [10]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split


df = pd.read_csv('H2O Quality Data/water_potability.csv')
df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


Data Analysis and Preprocessing

In [19]:
print(len(df)) # output 3276
print(df.shape) # output (3276, 10)

# remove all rows with NaN values
df.dropna(inplace=True)

scaler = StandardScaler() # or MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled.head()

# create new csv file with cleaned data, store in train folder
df_scaled.to_csv('H2O Quality Data/train/cleaned_water_potability.csv', index=False)
df = pd.read_csv('H2O Quality Data/train/cleaned_water_potability.csv')

2011
(2011, 10)


Data Splitting

In [24]:
# split data into temp_train and test sets
temp_train, test = train_test_split(df, test_size=0.2, random_state=42)

# split temp_train into final train and validate sets
train, validate = train_test_split(temp_train, test_size=0.1, random_state=42)

# check the sizes of the splits
print(f"Train set size: {len(train)}")
print(f"Validation set size: {len(validate)}")
print(f"Test set size: {len(test)}")

# save the splits as csv files
train.to_csv('H2O Quality Data/train/train.csv', index=False)
validate.to_csv('H2O Quality Data/validate/validate.csv', index=False)
test.to_csv('H2O Quality Data/test/test.csv', index=False)

Train set size: 1447
Validation set size: 161
Test set size: 403
